# Домашнее задание № 3. Исправление опечаток

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [1]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import textdistance
from tqdm.notebook import tqdm
import numpy as np
from collections import Counter
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

#### Ячейки из семинара:

In [2]:
bad = open('data/sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('data/correct_sents.txt', encoding='utf8').read().splitlines()

In [3]:
# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуацию на границах слов
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

In [4]:
corpus = open('data/wiki_data.txt', encoding='utf8').read()

In [5]:
vocab = Counter(re.findall('\w+', corpus.lower()))

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [6]:
def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1

In [7]:
N = sum(vocab.values())

def P(word, N=N):
    return np.log1p(vocab[word] / N)

In [8]:
def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

In [9]:
def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(topn)

#### Дописанная ячейка:

In [10]:
def get_closest_hybrid_match(text, X, vec, topn=5, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)
    
    # все кандидаты в виде словаря
    keys, values = [], []
    for close in closest:
        keys.append(close[0])
        values.append(close[1])
        dictionary = dict(zip(keys, values))
        
    # перебираем всех кандидатов и выбираем кандидата с наибольшей вероятностью        
    max_value = 0
    for k, v in dictionary.items():
        if v > max_value:
            max_value = v
            best = (k, v)
        elif v == max_value:
            choice = [best[0], k]
            # вот здесь выбираем
            best_word = max(choice, key=P)
            best = (best_word, dictionary[best_word])
            
    return best

In [11]:
%time
get_closest_hybrid_match('сонце', X, vec)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 7.63 µs


('солнце', 0.8333333333333334)

In [12]:
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab):
            pred = cashed.get(pair[1], get_closest_hybrid_match(pair[1], X, vec)[0][0])
            cashed[pair[1]] = pred
        else:
            pred = pair[1]
        
            
        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1
            
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1

  0%|          | 0/915 [00:00<?, ?it/s]

In [13]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.7928964482241121
0.0015527950310559005
0.09004249454461927


#### Стало хуже :( Это грустно. 

## 2.  Symspell (5 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Там к словам применяется только одна операция - удаление символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово   
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

#### Список правильных слов

In [14]:
correct_words = []
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    
    for pair in word_pairs:
        if pair[0] == pair[1]:
            correct_words.append(pair[0])

In [15]:
correct_words[:5]

['шпионское', 'устройство', 'такой', 'себе', 'гламурный']

In [16]:
print(len(set(correct_words)))

4175


In [17]:
correct_words = set(correct_words)

#### Словарь удалений

In [18]:
from collections import defaultdict
dict_all = defaultdict(lambda: defaultdict(list))

In [19]:
for word in correct_words:
    deletes = [word]
    if len(word) > 1:
        for c in range(len(word)): 
            word_minus_c = word[:c] + word[c + 1:]
            if word_minus_c not in deletes:
                deletes.append(word_minus_c)

    for item in deletes:
        dict_all[word][item].append(word)

In [20]:
for k, v in dict_all.items(): #просто посмотрим как выглядит словарь
    print(k, v)
    break

теперь defaultdict(<class 'list'>, {'теперь': ['теперь'], 'еперь': ['теперь'], 'тперь': ['теперь'], 'теерь': ['теперь'], 'тепрь': ['теперь'], 'тепеь': ['теперь'], 'тепер': ['теперь']})


Для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления.
Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово.

In [21]:
#функция -- для слова с опечаткой генерируются все варианты удаления
def get_wrong(word):

    deletes = [word]

    if len(word) > 1:
        for c in range(len(word)): 
            word_minus_c = word[:c] + word[c + 1:]
            if word_minus_c not in deletes:
                deletes.append(word_minus_c)
                
    return deletes

In [22]:
get_wrong('сонце')

['сонце', 'онце', 'снце', 'соце', 'соне', 'сонц']

In [23]:
def correction(word):
    variants = get_wrong(word) #генерируются все варианты удаления
    candidates = []
    for var in variants:
        for key, value in dict_all.items(): # имеем вложенный словарь
            for k in value.keys(): 
                if var == k:
                    candidates.append(*value[k]) #выбираются те, что есть в словаре удалений
    if not candidates:
        return word
    elif len(candidates) == 1:
        return candidates[0]
    else:
        return max(candidates, key=P) #наиболее вероятное правильное слово

In [24]:
correction('сонце')

'конце'

Почему (((((

In [25]:
P('сонце') > P('конце') #наверное вот поэтому (:

False

In [26]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        # чтобы два раза не исправлять одно и тоже слово - закешируем его
        # перед тем как считать исправление проверим нет ли его в кеше
        
        predicted = cashed.get(pair[1], correction(pair[1]))
        cashed[pair[1]] = predicted
        
        
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1

  0%|          | 0/915 [00:00<?, ?it/s]

In [27]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.6249124562281141
0.25388198757763975
0.320202136212243


In [28]:
%%time
correction('солнвце')

CPU times: user 12.7 ms, sys: 80 µs, total: 12.8 ms
Wall time: 12.6 ms


'солнце'

Как-то грустно.

## *3. Чтение. (2 балла)

Прочитайте эту главу в книге Speech and Language Processing - https://web.stanford.edu/~jurafsky/slp3/2.pdf .
Ответьте на следующий вопрос:

1. Что такое Byte-Pair Encoding (опишите по-русски, как минимум 10 предложениями)?

*Это задание не связано напрямую с исправлением опечаток, но это важная тема, к которой мы вернемся в конце курса

Byte-Pair Encoding -- вариант токенизации текста (вместо определения токенов как слов / символов). 

Это самый универсальный и широко применяемый способ токенизации сейчас (с некоторыми вариациями и улучшениями, но концептуально во всех SOTA моделях). 

Чем он хорош: позволяет решать проблему неизвестных слов. Если наши данные содержат слова low, new, newer, но не *lower*, то наша модель не будет знать, что с этим словом делать.

BPE-токенизатор автоматически определяет набор токенов, который включает "подслова".

Подслова могут быть произвольными подстроками (low) или несущими значение единицами (морфемы -est или -er).

Как это работает: сначала BPE-tokenizer учится на корпусе, потом разбивает слова на подслова. 

Как он учится: есть корпус --> там выбирает два символа, которые наиболее часто соседствуют (например, ‘A’, ‘B’), затем добавляет новый символ ‘AB’ в словарь и заменяет каждые соседствующие ’A’ и ’B’ в корпусе на ‘AB’. Он продолжает подсчитывать и объединять, создавая новые все более длинные подслова. Алгоритм запускается на отдельных словах (поэтому они сначала разделяются пробелами + у каждого слова есть спец. символ конца слова -- "_").

Результат: большинство слов будут представлены как есть (new_), редкие и неизвестные слова должны быть представлены по частям (low+er_).